In [5]:
from pynq import Overlay
overlay=Overlay("./ready_to_test/pro_int_trans.bit")
pro_int_trans=overlay.pro_int_trans_0

import struct
from time import *
import matplotlib.pyplot as plt
import numpy as np
import mpl_toolkits.axisartist as axisartist
import math

XPRO_INT_TRANS_AXILITES_ADDR_C_INT_DATA=0x10
XPRO_INT_TRANS_AXILITES_ADDR_C_INT_CTRL=0x14
XPRO_INT_TRANS_AXILITES_ADDR_X_INT_DATA=0x18
XPRO_INT_TRANS_AXILITES_ADDR_Y_INT_DATA=0x20
XPRO_INT_TRANS_AXILITES_ADDR_K_INT_DATA=0x28
XPRO_INT_TRANS_AXILITES_ADDR_L_INT_DATA=0x30
XPRO_INT_TRANS_AXILITES_ADDR_M_INT_DATA=0x38
XPRO_INT_TRANS_AXILITES_ADDR_C1_INT_DATA=0x40
XPRO_INT_TRANS_AXILITES_ADDR_A1_INT_DATA=0x48
XPRO_INT_TRANS_AXILITES_ADDR_D1_INT_DATA=0x50


def float_to_hex(f):
    return struct.unpack('<I', struct.pack('<f', f))[0]
def hex_to_float(h):
    i = int(h,16)
    return struct.unpack('<f',struct.pack('<I', i))[0]

####一个方程
def f_fpga(x,y,k,l,m,c1,a1,d1):
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_X_INT_DATA,float_to_hex(x))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_Y_INT_DATA,float_to_hex(y))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_K_INT_DATA,float_to_hex(k))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_L_INT_DATA,float_to_hex(l))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_M_INT_DATA,float_to_hex(m))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_C1_INT_DATA,float_to_hex(c1))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_A1_INT_DATA,float_to_hex(a1))
    pro_int_trans.write(XPRO_INT_TRANS_AXILITES_ADDR_D1_INT_DATA,float_to_hex(d1))
    return hex_to_float(hex(pro_int_trans.read(XPRO_INT_TRANS_AXILITES_ADDR_C_INT_DATA)))

def f_cpu(x,y,k,l,m,c1,a1,d1):
    return (k*x+l*y+m*x*y+c1)*math.exp(a1*x+d1*y)

def Euler_cpu(k,l,m,c1,a1,d1,a,b,t0,u0,h,n):
    u[0]=u0;
    for i in range(1,n+1):
        u[i]=u[i-1]+h*f_fpga(t0+i*h,u[i-1],k,l,m,c1,a1,d1);

def Euler_fpga(k,l,m,c1,a1,d1,a,b,t0,u0,h,n):
    u[0]=u0;
    for i in range(1,n+1):
        u[i]=u[i-1]+h*f_fpga(t0+i*h,u[i-1],k,l,m,c1,a1,d1);

def Runge_Kutta_cpu(k,l,m,c1,a1,d1,a,b,t0,u0,h,n):
    u[0]=u0;
    for i in range(1,n+1):
        k1=h*f_cpu(t0+i*h,u[i-1],k,l,m,c1,a1,d1);
        k2=h*f_cpu(t0+i*h+0.5*h,u[i-1]+0.5*k1,k,l,m,c1,a1,d1);
        k3=h*f_cpu(t0+i*h+0.5*h,u[i-1]+0.5*k2,k,l,m,c1,a1,d1);
        k4=h*f_cpu(t0+i*h+h,u[i-1]+k3,k,l,m,c1,a1,d1);
        u[i]=u[i-1]+1.0/6.0*(k1+2*k2+2*k3+k4);

def Runge_Kutta_fpga(k,l,m,c1,a1,d1,a,b,t0,u0,h,n):
    u[0]=u0;
    for i in range(1,n+1):
        k1=h*f_fpga(t0+i*h,u[i-1],k,l,m,c1,a1,d1);
        k2=h*f_fpga(t0+i*h+0.5*h,u[i-1]+0.5*k1,k,l,m,c1,a1,d1);
        k3=h*f_fpga(t0+i*h+0.5*h,u[i-1]+0.5*k2,k,l,m,c1,a1,d1);
        k4=h*f_fpga(t0+i*h+h,u[i-1]+k3,k,l,m,c1,a1,d1);
        u[i]=u[i-1]+1.0/6.0*(k1+2*k2+2*k3+k4);



In [26]:
print("\n*****************************************")
func_num=int(input("Please enter the number of equations (groups) to be solved (1~4):"))


###### 1_function
if func_num==1:
    print("the unsolved function is: \n\
    (1)	y'=f(x,y) \n\
    (2)	f(x,y)=(kx+ly+mxy+c1)*exp(a1*x+d1*y) \n\
    (3)	a<=x<=b \n\
    (4)	y(t0)=u0 \n\
    (5)	step value h")
    print("now please input the parameter k,l,m,c1,a1,d1,a,b,t0 u0 and h:\n")
    k,l,m,c1=map(float,input("please input k l m and c1:").split())
    a1,d1=map(float,input("please input a1 and d1:").split())
    a,b=map(float,input("please input a and b:").split())
    while(a>b):
        a,b=map(float,input("input [a b] error! please reinput the terminal value:[a,b] like:  0 1:").split())

    t0,u0=map(float,input("please input t0 and u0:").split())
    while (a>t0)or(b<t0) :
        print("input t0 =",t0,"is not inside [",a," ,",b,"]!")
        t0,u0=map(float,input("please reinput the initial value of x and y: t0,u0 =: ").split())

    h=input("please input h:")
    h=float(h)
    if h>b-a:
        print("WARNNING! input h is bigger than b-a,we put the h = b - a =",b-a)
        h=b-a;
    n=int((b-a)/h)

    solve_function=0;
    solve_function=input("please input solving method: 0 for Runge-Kutta method,1 for Euler method:");


    u=[0 for i in range(n+1)]
    t=np.linspace(a, b, n+1)
    print("*****************************************\n")
    print("\n*****************************************")
    print("the unsolved function is: \n\
    (1)	y'=f(x,y) \n\
    (2)	f(x,y)=(",k,"x+",l,"y+",m,"xy+",c1,")*exp(",a1,"x+",d1,"y) \n\
    (3)	",a,"<=x<=",b," \n\
    (4)	y(",t0,")=",u0," \n\
    (5)	step value is ",h);
    if int(solve_function)==0:
        print("solving method is Runge-Kutta method")
    else:
        print("solving method is Euler method\n")
    print("*****************************************")
    print("solving...")




    if int(solve_function)==0:
        time_cpu_start = time()
        Runge_Kutta_cpu(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_cpu_end=time()

        time_fpga_start = time()
        Runge_Kutta_fpga(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_fpga_end=time()

        print("\n*****************************************");
        solution_time_cpu = time_cpu_end - time_cpu_start
        solution_time_fpga = time_fpga_end - time_fpga_start
        speed_up_rate=solution_time_fpga/solution_time_cpu
        print("total solution time on CPU is %.5f"%solution_time_cpu,"s")
        print("total solution time when using fpga speeding up is %.5f"%solution_time_fpga,"s")
        print("speed up rate is %.3f times faster"%speed_up_rate)
        print("the solution is:");
        print("x	y");
        if n<20:
            for i in range(n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        else:
            for i in range(10):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
            print("......","\t","......")
            for i in range(n-9,n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        print("\n")

    else:
        time_cpu_start = time()
        Euler_cpu(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_cpu_end=time()

        time_fpga_start = time()
        Euler_fpga(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_fpga_end=time()

        print("\n*****************************************");
        solution_time_cpu = time_cpu_end - time_cpu_start
        solution_time_fpga = time_fpga_end - time_fpga_start
        speed_up_rate=solution_time_fpga/solution_time_cpu
        print("total solution time on CPU is %.5f"%solution_time_cpu,"s")
        print("total solution time when using fpga speeding up is %.5f"%solution_time_fpga,"s")
        print("speed up rate is %.3f times faster"%speed_up_rate)
        print("the solution is:");
        print("x	y");
        if n<20:
            for i in range(n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        else:
            for i in range(10):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
            print("......","\t","......")
            for i in range(n-9,n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        print("\n")


    fig = plt.figure(figsize=(8, 8))
    #使用axisartist.Subplot方法创建一个绘图区对象ax
    ax = axisartist.Subplot(fig, 111)
    #将绘图区对象添加到画布中
    fig.add_axes(ax)
    #通过set_visible方法设置绘图区所有坐标轴隐藏
    ax.axis[:].set_visible(False)

    #ax.new_floating_axis代表添加新的坐标轴
    ax.axis["x"] = ax.new_floating_axis(0,0)
    #给x坐标轴加上箭头
    ax.axis["x"].set_axisline_style("->", size = 1.0)
    #添加y坐标轴，且加上箭头
    ax.axis["y"] = ax.new_floating_axis(1,0)
    ax.axis["y"].set_axisline_style("-|>", size = 1.0)
    #设置x、y轴上刻度显示方向
    ax.axis["x"].set_axis_direction("top")
    ax.axis["y"].set_axis_direction("right")
    plt.scatter(t, u, color='r', marker='.')
    plt.show()
    print("\n")
    print("\n")
    print("\n")
    
    #########2 function groups:
elif func_num==2:
    print("the unsolved function is: \n\
    (1)	y1'=f1(x,y1,y2),y2'=f2(x,y1,y2) \n\
    (2)	f1(x,y1,y2)=(k1*x+l1*y1+m1*y2+p1*x*y1+q1*x*y2+r1*x*y1*y2+s1)*exp(u1*x+v1*y1+w1*y2) \n\
    (3)	f2(x,y1,y2)=(k2*x+l2*y1+m2*y2+p2*x*y1+q2*x*y2+r2*x*y1*y2+s2)*exp(u2*x+v2*y1+w2*y2) \n\
    (4)	a<=x<=b \n\
    (5)	y1(t0)=u0_1,y2(t0)=u0_2 \n\
    (6)	step value == h")
 
    print("now please input the parameter k1,l1,m1,p1,q1,r1,s1,u1,v1,w1,k2,l2,m2,p2,q2,r2,s2,u2,v2,w2,a,b,t0 u0_1,u0_2 and h:\n")
    k1,l1,m1,p1,q1,r1,s1=map(float,input("please input k1,l1,m1,p1,q1,r1,s1:").split())
    u1,v1,w1=map(float,input("please input u1,v1,w1:").split())
    k2,l2,m2,p2,q2,r2,s2=map(float,input("please input k2,l2,m2,p2,q2,r2,s2:").split())
    u2,v2,w2=map(float,input("please input u2,v2,w2:").split())
    
    a,b=map(float,input("please input a and b:").split())
    while(a>b):
        a,b=map(float,input("input [a b] error! please reinput the terminal value:[a,b] like:  0 1:").split())

    t0,u0_1,u0_2=map(float,input("please input t0 and u0_1,u0_2:").split())
    while (a>t0)or(b<t0) :
        print("input t0 =",t0,"is not inside [",a," ,",b,"]!")
        t0,u0_1,u0_2=map(float,input("please reinput the initial value of x, y1 and y2: t0,u0_1,u0_2 =: ").split())

    h=float(input("please input h:"))
    if h>b-a:
        print("WARNNING! input h is bigger than b-a,we put the h = b - a =",b-a)
        h=b-a;
    n=int((b-a)/h)

    solve_function=0;
    solve_function=input("please input solving method: 0 for Runge-Kutta method,1 for Euler method:");


    u_1=[0 for i in range(n+1)]
    u_2=[0 for i in range(n+1)]
    
    t=np.linspace(a, b, n+1)
    print("*****************************************\n")
    print("\n*****************************************")
    print("the unsolved function is: \n\
    (1)	y1'=f1(x,y1,y2), y2'=f2(x,y1,y2) \n\
    (2)	f1(x,y1,y2)=(",k1,"*x+",l1,"*y1+",m1,"*y2+",p1,"*x*y1+",q1,"*x*y2+",r1,"*x*y1*y2+",s1,")*exp(",u1,"*x+",v1,"*y1+",w1,"*y2) \n\
    (3)	f2(x,y1,y2)=(",k2,"*x+",l2,"*y1+",m2,"*y2+",p2,"*x*y1+",q2,"*x*y2+",r2,"*x*y1*y2+",s2,")*exp(",u2,"*x+",v2,"*y1+",w2,"*y2) \n\
    (4)	",a,"<=x<=",b," \n\
    (5)	y1(",t0,")=",u0_1,",y2(",t0,")=",u0_2," \n\
    (6)	step value = ",h)
    if int(solve_function)==0:
        print("solving method is Runge-Kutta method")
    else:
        print("solving method is Euler method")
    print("*****************************************\n")
    print("solving...")




    if int(solve_function)==0:
        time_cpu_start = time()
        Runge_Kutta_cpu(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_cpu_end=time()

        time_fpga_start = time()
        Runge_Kutta_fpga(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_fpga_end=time()

        print("\n*****************************************");
        solution_time_cpu = time_cpu_end - time_cpu_start
        solution_time_fpga = time_fpga_end - time_fpga_start
        speed_up_rate=solution_time_fpga/solution_time_cpu
        print("total solution time on CPU is %.5f"%solution_time_cpu,"s")
        print("total solution time when using fpga speeding up is %.5f"%solution_time_fpga,"s")
        print("speed up rate is %.3f times faster"%speed_up_rate)
        print("the solution is:");
        print("x	y");
        if n<20:
            for i in range(n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        else:
            for i in range(10):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
            print("......","\t","......")
            for i in range(n-9,n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        print("\n")

    else:
        time_cpu_start = time()
        Euler_cpu(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_cpu_end=time()

        time_fpga_start = time()
        Euler_fpga(k,l,m,c1,a1,d1,a,b,t0,u0,h,n);
        time_fpga_end=time()

        print("\n*****************************************");
        solution_time_cpu = time_cpu_end - time_cpu_start
        solution_time_fpga = time_fpga_end - time_fpga_start
        speed_up_rate=solution_time_fpga/solution_time_cpu
        print("total solution time on CPU is %.5f"%solution_time_cpu,"s")
        print("total solution time when using fpga speeding up is %.5f"%solution_time_fpga,"s")
        print("speed up rate is %.3f times faster"%speed_up_rate)
        print("the solution is:");
        print("x	y");
        if n<20:
            for i in range(n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        else:
            for i in range(10):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
            print("......","\t","......")
            for i in range(n-9,n+1):
                print("%.3f"%(a+i*h),"\t","%.3f"%u[i]);
        print("\n")


    fig = plt.figure(figsize=(8, 8))
    #使用axisartist.Subplot方法创建一个绘图区对象ax
    ax = axisartist.Subplot(fig, 111)
    #将绘图区对象添加到画布中
    fig.add_axes(ax)
    #通过set_visible方法设置绘图区所有坐标轴隐藏
    ax.axis[:].set_visible(False)

    #ax.new_floating_axis代表添加新的坐标轴
    ax.axis["x"] = ax.new_floating_axis(0,0)
    #给x坐标轴加上箭头
    ax.axis["x"].set_axisline_style("->", size = 1.0)
    #添加y坐标轴，且加上箭头
    ax.axis["y"] = ax.new_floating_axis(1,0)
    ax.axis["y"].set_axisline_style("-|>", size = 1.0)
    #设置x、y轴上刻度显示方向
    ax.axis["x"].set_axis_direction("top")
    ax.axis["y"].set_axis_direction("right")
    plt.scatter(t, u, color='r', marker='.')
    plt.show()
    print("\n")
    print("\n")
    print("\n")
    


*****************************************


Please enter the number of equations (groups) to be solved (1~4): 2


the unsolved function is: 
    (1)	y1'=f1(x,y1,y2),y2'=f2(x,y1,y2) 
    (2)	f1(x,y1,y2)=(k1*x+l1*y1+m1*y2+p1*x*y1+q1*x*y2+r1*x*y1*y2+s1)*exp(u1*x+v1*y1+w1*y2) 
    (3)	f2(x,y1,y2)=(k2*x+l2*y1+m2*y2+p2*x*y1+q2*x*y2+r2*x*y1*y2+s2)*exp(u2*x+v2*y1+w2*y2) 
    (4)	a<=x<=b 
    (5)	y1(t0)=u0_1,y2(t0)=u0_2 
    (6)	step value == h
now please input the parameter k1,l1,m1,p1,q1,r1,s1,u1,v1,w1,k2,l2,m2,p2,q2,r2,s2,u2,v2,w2,a,b,t0 u0_1,u0_2 and h:



please input k1,l1,m1,p1,q1,r1,s1: 1 2 3 4 5 6 7.1
please input u1,v1,w1: 8 9 10
please input k2,l2,m2,p2,q2,r2,s2: 7 6 5 4 3 2 1.1
please input u2,v2,w2: 0.9 0.8 0.7
please input a and b: 0.6 1
please input t0 and u0_1,u0_2: -10291 29931 19293


input t0 = -10291.0 is not inside [ 0.6  , 1.0 ]!


please reinput the initial value of x, y1 and y2: t0,u0_1,u0_2 =:  0.8 81283 -123919
please input h: 0.0001
please input solving method: 0 for Runge-Kutta method,1 for Euler method: 0


*****************************************


*****************************************
the unsolved function is: 
    (1)	y1'=f1(x,y1,y2), y2'=f2(x,y1,y2) 
    (2)	f1(x,y1,y2)=( 1.0 *x+ 2.0 *y1+ 3.0 *y2+ 4.0 *x*y1+ 5.0 *x*y2+ 6.0 *x*y1*y2+ 7.1 )*exp( 8.0 *x+ 9.0 *y1+ 10.0 *y2) 
    (3)	f2(x,y1,y2)=( 7.0 *x+ 6.0 *y1+ 5.0 *y2+ 4.0 *x*y1+ 3.0 *x*y2+ 2.0 *x*y1*y2+ 1.1 )*exp( 0.9 *x+ 0.8 *y1+ 0.7 *y2) 
    (4)	 0.6 <=x<= 1.0  
    (5)	y1( 0.8 )= 81283.0 ,y2( 0.8 )= -123919.0  
    (6)	step value =  0.0001
solving method is Runge-Kutta method

*****************************************

solving...


IndexError: list assignment index out of range

In [32]:
!uptime -p

up 3 hours, 17 minutes
